In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
import time

In [ ]:
# Set up headless Chrome
options = Options()
options.add_argument("--headless=new")  # Use "--headless" for older Chrome
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

In [ ]:
url = "https://www.uniqlo.com/de/de/products/E478984-000/00?colorDisplayCode=00&sizeDisplayCode=004"

In [ ]:
driver = webdriver.Chrome(options=options)


# Retryable size extraction
def get_size_info_with_retry(driver, index, max_retries=3):
    for attempt in range(max_retries):
        try:
            # Refetch fresh wrapper every time
            wrappers = driver.find_elements(By.CSS_SELECTOR, 'div.size-chip-wrapper')
            wrapper = wrappers[index]

            button = wrapper.find_element(By.CSS_SELECTOR, 'button[data-testid="ITOChip"]')
            label_div = button.find_element(By.CSS_SELECTOR, '[data-testid="ITOTypography"]')
            size_label = driver.execute_script("return arguments[0].textContent;", label_div).strip()

            # Availability
            has_strike = len(wrapper.find_elements(By.CSS_SELECTOR, 'div.strike')) > 0
            available = not has_strike

            return size_label, available

        except StaleElementReferenceException as e:
            print(f"Retry {attempt + 1} due to stale element")
            time.sleep(0.5)

    return "N/A", False

try:
    driver.get(url)

    # Wait for size elements to load
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'div.size-chip-wrapper'))
    )

    # Get the number of size wrappers (do this only once!)
    total = len(driver.find_elements(By.CSS_SELECTOR, 'div.size-chip-wrapper'))

    for i in range(total):
        size_label, available = get_size_info_with_retry(driver, i)
        print(f"Size: {size_label} | Available: {available}")


finally:
    driver.quit()
    print("done")